In [2]:
!pip install -r requirements.txt


  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.6.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.18-py312-none-any.whl.metadata (7.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 25.6 MB/s eta 0:00:00a 0:00:01
Using cached dill-0.4.0-py3-none-any.whl (119 kB)
Using cached multiprocess-0.70.18-py312-none-any.whl (150 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 34.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.2 MB/s eta 0:00:00
Using cached xxhash-3.6.0-cp312-cp312-macosx_11_0_arm64.whl (30 kB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
ERROR: 

In [ ]:
#@title Install dependencies and mount Google Drive


import os
from pathlib import Path

from google.colab import drive
drive.mount('/content/drive')

BASE_DIR = Path("/content/drive/MyDrive/emergence_gpt2_modularity") #or change this to whatever works for you. Will create new directory if needed
BASE_DIR.mkdir(parents=True, exist_ok=True)
print("Base dir:", BASE_DIR)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 120.0 MB/s eta 0:00:00
Mounted at /content/drive
Base dir: /content/drive/MyDrive/emergence_gpt2_modularity


In [ ]:
#@title Configuration

import torch
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

MODEL_NAMES = [
    "gpt2",
    "gpt2-medium",
    "gpt2-large",
    "gpt2-xl",
]


MAX_TOKENS = 20000   # total token positions sampled per model (for activations & biclustering)
SEQ_LEN    = 128     # context window per sequence during forward passes

# Biclustering grid to try for best bicluster options
ROW_KS = [6, 9, 12]
COL_KS = [6, 9, 12]

# Z-score null sampling
N_SHUFFLES = 10

# Random seed for reproducibility
RNG_SEED = 42
np.random.seed(RNG_SEED)
torch.manual_seed(RNG_SEED)

print("Config set.")


Using device: cuda
Config set.


In [ ]:
from datasets import load_dataset

# === Active: Wikipedia via wikimedia/wikipedia (English) ===
ds = load_dataset(
    "wikimedia/wikipedia",
    "20231101.en",
    split="train",
    streaming=True,
)
text_col = "text"

print(ds)


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

IterableDataset({
    features: ['id', 'url', 'title', 'text'],
    num_shards: 41
})


In [ ]:
#@title Build a fixed token stream (shared across all models)

from transformers import AutoTokenizer
from tqdm.auto import tqdm

tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

TARGET_TOKENS = MAX_TOKENS * 2
texts = []
num_docs = 0

print("Collecting text from dataset...")
for example in tqdm(ds):
    txt = example.get(text_col, None)
    if not txt or not isinstance(txt, str):
        continue
    texts.append(txt)
    num_docs += 1
    if num_docs >= 2000:  # cap docs to avoid going wild; adjust if needed
        break

print(f"Collected {len(texts)} documents.")

print("Tokenizing...")
all_ids = []
for t in tqdm(texts):
    ids = tokenizer(t, add_special_tokens=False)["input_ids"]
    all_ids.extend(ids)

all_ids = np.array(all_ids[:TARGET_TOKENS], dtype=np.int32)
print("Total tokens in fixed stream:", len(all_ids))

# Save token stream to Drive so we can reuse or work with
tok_path = BASE_DIR / "dataset_tokens.npz"
np.savez(tok_path, input_ids=all_ids)
print("Saved tokens to:", tok_path)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

0it [00:00, ?it/s]

Collected 2000 documents.
Tokenizing...


  0%|          | 0/2000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (8668 > 1024). Running this sequence through the model will result in indexing errors


Total tokens in fixed stream: 40000
Saved tokens to: /content/drive/MyDrive/emergence_gpt2_modularity/dataset_tokens.npz


In [ ]:
#@title Create sequences from token stream

def make_sequences_from_stream(input_ids, seq_len):
    """Chunk a 1D array of token IDs into sequences of length seq_len."""
    total = len(input_ids)
    usable = (total // seq_len) * seq_len
    ids = input_ids[:usable]
    ids = ids.reshape(-1, seq_len)
    return ids

data = np.load(BASE_DIR / "dataset_tokens.npz")["input_ids"]
seqs = make_sequences_from_stream(data, SEQ_LEN)
print("Sequences shape:", seqs.shape)


Sequences shape: (312, 128)


In [ ]:
#@title Collect activations for each GPT-2 model
#Note this will take up running where left off if execution is interrupted

from transformers import AutoModelForCausalLM
from tqdm.auto import tqdm

def collect_activations_for_model(model_name, seqs, max_tokens, device):
    """
    Returns:
      layer_mats: dict[layer_idx] -> np.ndarray of shape (max_tokens, H)
      H: hidden size
      L: number of layers
    """
    print(f"\n=== Collecting activations for {model_name} ===")
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.to(device)
    model.eval()

    # Check config
    n_layer = model.config.n_layer
    hidden_size = model.config.n_embd
    print(f"Layers: {n_layer}, hidden size: {hidden_size}")

    # Preallocate per-layer buffers
    layer_mats = {
        ell: np.zeros((max_tokens, hidden_size), dtype=np.float32)
        for ell in range(n_layer)
    }
    counts = {ell: 0 for ell in range(n_layer)}

    with torch.no_grad():
        for i in tqdm(range(len(seqs)), desc=f"{model_name} batches"):
            batch_ids = torch.tensor(seqs[i:i+1], dtype=torch.long, device=device)
            out = model(batch_ids, output_hidden_states=True)
            # hidden_states: tuple length n_layer+1 (including embedding layer)
            hidden_states = out.hidden_states

            # For each layer 0..n_layer-1 (skip embedding)
            for ell in range(n_layer):
                hs = hidden_states[ell+1].detach().cpu().numpy()
                hs = hs.reshape(-1, hidden_size)

                need = max_tokens - counts[ell]
                if need <= 0:
                    continue
                take = min(need, hs.shape[0])
                layer_mats[ell][counts[ell]:counts[ell]+take, :] = hs[:take]
                counts[ell] += take

            # Stop early if all layers filled
            if all(counts[ell] >= max_tokens for ell in range(n_layer)):
                break

    # Trim
    for ell in range(n_layer):
        if counts[ell] < max_tokens:
            layer_mats[ell] = layer_mats[ell][:counts[ell], :]
            print(f"Layer {ell}: only {counts[ell]} tokens collected.")
        else:
            print(f"Layer {ell}: collected {max_tokens} tokens.")

    # Save to Drive to use later if needed
    model_dir = BASE_DIR / model_name.replace("/", "_")
    model_dir.mkdir(parents=True, exist_ok=True)
    meta = {
        "model_name": model_name,
        "n_layer": n_layer,
        "hidden_size": hidden_size,
        "max_tokens": max_tokens,
    }
    np.savez(model_dir / "meta.npz", **meta)
    for ell in range(n_layer):
        np.savez_compressed(model_dir / f"activations_layer{ell:02d}.npz",
                            activations=layer_mats[ell])

    print(f"Saved activations to {model_dir}")
    del model
    torch.cuda.empty_cache()
    return layer_mats, hidden_size, n_layer

for model_name in [MODEL_NAMES]:
# for model_name in ["gpt2-xl"]:
    _ = collect_activations_for_model(model_name, seqs, MAX_TOKENS, device)



=== Collecting activations for gpt2-xl ===


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Layers: 48, hidden size: 1600


gpt2-xl batches:   0%|          | 0/312 [00:00<?, ?it/s]

Layer 0: collected 20000 tokens.
Layer 1: collected 20000 tokens.
Layer 2: collected 20000 tokens.
Layer 3: collected 20000 tokens.
Layer 4: collected 20000 tokens.
Layer 5: collected 20000 tokens.
Layer 6: collected 20000 tokens.
Layer 7: collected 20000 tokens.
Layer 8: collected 20000 tokens.
Layer 9: collected 20000 tokens.
Layer 10: collected 20000 tokens.
Layer 11: collected 20000 tokens.
Layer 12: collected 20000 tokens.
Layer 13: collected 20000 tokens.
Layer 14: collected 20000 tokens.
Layer 15: collected 20000 tokens.
Layer 16: collected 20000 tokens.
Layer 17: collected 20000 tokens.
Layer 18: collected 20000 tokens.
Layer 19: collected 20000 tokens.
Layer 20: collected 20000 tokens.
Layer 21: collected 20000 tokens.
Layer 22: collected 20000 tokens.
Layer 23: collected 20000 tokens.
Layer 24: collected 20000 tokens.
Layer 25: collected 20000 tokens.
Layer 26: collected 20000 tokens.
Layer 27: collected 20000 tokens.
Layer 28: collected 20000 tokens.
Layer 29: collected 2000

In [ ]:
#@title Metrics helpers: normalization, R², Z-score, specialization

from sklearn.cluster import SpectralBiclustering, SpectralCoclustering

def normalize_activations(M):
    """Z-score per neuron, then abs."""
    M = M.astype(np.float32)
    mean = M.mean(axis=0, keepdims=True)
    std  = M.std(axis=0, keepdims=True) + 1e-6
    Z = (M - mean) / std
    return np.abs(Z), mean, std

def block_model_reconstruction(A, row_labels, col_labels):
    """Return block-mean reconstruction given biclustering labels."""
    R = row_labels.max() + 1
    C = col_labels.max() + 1
    H, W = A.shape
    # Compute block means
    block_means = np.zeros((R, C), dtype=np.float32)
    counts = np.zeros((R, C), dtype=np.int64)
    for i in range(H):
        r = row_labels[i]
        for j in range(W):
            c = col_labels[j]
            block_means[r, c] += A[i, j]
            counts[r, c] += 1
    counts[counts == 0] = 1
    block_means /= counts
    # Reconstruct
    A_hat = np.zeros_like(A)
    for i in range(H):
        r = row_labels[i]
        A_hat[i, :] = block_means[r, col_labels]
    return A_hat, block_means

def r2_block_model(A, row_labels, col_labels):
    """Compute R² of block-mean model."""
    A_hat, _ = block_model_reconstruction(A, row_labels, col_labels)
    mean = A.mean()
    tss = np.sum((A - mean)**2)
    rss = np.sum((A - A_hat)**2)
    if tss <= 1e-12:
        return 0.0
    return float(1.0 - rss / tss)

def block_stat_mean_abs(block_means):
    """Simple summary statistic: mean of block means."""
    return float(block_means.mean())

def enrichment_z_score(A, row_labels, col_labels, n_shuffles=10, rng=None):
    """Z-score of block mean statistic vs shuffled row labels."""
    if rng is None:
        rng = np.random.RandomState(RNG_SEED)

    # observed statistic
    _, block_means = block_model_reconstruction(A, row_labels, col_labels)
    S_obs = block_stat_mean_abs(block_means)

    # null distribution via shuffling row labels
    null = []
    for _ in range(n_shuffles):
        shuffled_rows = rng.permutation(row_labels)
        _, bm_null = block_model_reconstruction(A, shuffled_rows, col_labels)
        null.append(block_stat_mean_abs(bm_null))

    null = np.array(null, dtype=np.float32)
    mu = null.mean()
    sigma = null.std() + 1e-6
    Z = (S_obs - mu) / sigma
    return float(Z), S_obs, mu, sigma



In [ ]:
#@title Biclustering search

def run_biclustering_grid(A, row_ks, col_ks, n_shuffles=10, method="spectral"):
    """
    A: (tokens, neurons) normalized nonnegative activation matrix.
    Returns:
        best_result: dict with keys:
          'row_labels', 'col_labels', 'R2', 'Z', 'row_k', 'col_k', 'stat_obs', 'null_mu', 'null_sigma'
        all_results: list of results
    """
    H, W = A.shape
    results = []
    best = None

    for kr in row_ks:
        for kc in col_ks:
            print(f"  Trying (K_row={kr}, K_col={kc})...")
            if method == "spectral":
                model = SpectralBiclustering(n_clusters=(kr, kc), random_state=RNG_SEED)
            else:
                model = SpectralCoclustering(n_clusters=(kr, kc), random_state=RNG_SEED)
            model.fit(A)
            row_labels = model.row_labels_
            col_labels = model.column_labels_

            R2 = r2_block_model(A, row_labels, col_labels)
            Z, S_obs, mu, sigma = enrichment_z_score(A, row_labels, col_labels, n_shuffles=n_shuffles)

            res = {
                "row_k": kr,
                "col_k": kc,
                "R2": R2,
                "Z": Z,
                "stat_obs": S_obs,
                "null_mu": mu,
                "null_sigma": sigma,
                "row_labels": row_labels,
                "col_labels": col_labels,
            }
            results.append(res)
            print(f"    -> R²={R2:.3f}, Z={Z:.2f}")

            if (best is None) or (R2 > best["R2"]):
                best = res

    print("Best config:", best["row_k"], best["col_k"], "R²=", best["R2"], "Z=", best["Z"])
    return best, results


In [ ]:
import pandas as pd


def analyze_model_layers(model_name, resume=True):
    print(f"\n=== Analyzing model {model_name} ===")
    mdir = BASE_DIR / model_name.replace("/", "_")
    meta = np.load(mdir / "meta.npz")
    n_layer = int(meta["n_layer"])
    hidden_size = int(meta["hidden_size"])
    max_tokens = int(meta["max_tokens"])

    records = []
    cluster_dir = mdir / "clusters"
    cluster_dir.mkdir(exist_ok=True)

    metrics_path = mdir / "layer_metrics.csv"

    # If everything looks finished and we're resuming, just load the CSV
    if resume and metrics_path.exists():
        existing_layers = {
            int(p.stem.split("_")[0].replace("layer", ""))
            for p in cluster_dir.glob("layer*_best_cluster.npz")
        }
        if len(existing_layers) == n_layer:
            print("  All layers already processed; loading metrics from CSV.")
            df = pd.read_csv(metrics_path)
            return df

    for ell in range(n_layer):
        cluster_file = cluster_dir / f"layer{ell:02d}_best_cluster.npz"

        if resume and cluster_file.exists():
            print(f"\nLayer {ell}: already done, loading cached results.")
            cached = np.load(cluster_file)

            records.append({
                "model": model_name,
                "layer": ell,
                "R2": float(cached["R2"]),
                "Z": float(cached["Z"]),
                "row_k": int(cached["row_k"]),
                "col_k": int(cached["col_k"]),
                "spec_mean": float(cached["spec_mean"]),
                "spec_median": float(cached["spec_median"]),
            })
            continue

        print(f"\nLayer {ell}:")
        data = np.load(mdir / f"activations_layer{ell:02d}.npz")["activations"]
        print("  raw shape:", data.shape)

        A, mean, std = normalize_activations(data)
        print("  normalized shape:", A.shape)

        best, all_results = run_biclustering_grid(
            A, ROW_KS, COL_KS,
            n_shuffles=N_SHUFFLES,
            method="spectral"
        )

        # Specialization
        S = specialization_entropy(A)
        S_mean = float(S.mean())
        S_median = float(np.median(S))

        # Save best clustering labels & metrics so we can resume later if needed
        np.savez_compressed(
            cluster_file,
            row_labels=best["row_labels"],
            col_labels=best["col_labels"],
            row_k=best["row_k"],
            col_k=best["col_k"],
            R2=best["R2"],
            Z=best["Z"],
            spec_mean=S_mean,
            spec_median=S_median,
        )

        records.append({
            "model": model_name,
            "layer": ell,
            "R2": best["R2"],
            "Z": best["Z"],
            "row_k": best["row_k"],
            "col_k": best["col_k"],
            "spec_mean": S_mean,
            "spec_median": S_median,
        })

    # Make sure layers are in order even if some were resumed out of order
    df = pd.DataFrame(records).sort_values("layer").reset_index(drop=True)
    df.to_csv(metrics_path, index=False)
    print("Saved metrics to:", metrics_path)
    return df


all_metrics = []
for model_name in MODEL_NAMES:
    df_m = analyze_model_layers(model_name)
    all_metrics.append(df_m)

metrics_df = pd.concat(all_metrics, ignore_index=True)
metrics_df.to_csv(BASE_DIR / "combined_layer_metrics.csv", index=False)
print("Saved combined metrics to:", BASE_DIR / "combined_layer_metrics.csv")



=== Analyzing model gpt2 ===
  All layers already processed; loading metrics from CSV.

=== Analyzing model gpt2-medium ===
  All layers already processed; loading metrics from CSV.

=== Analyzing model gpt2-large ===

Layer 0: already done, loading cached results.

Layer 1: already done, loading cached results.

Layer 2: already done, loading cached results.

Layer 3: already done, loading cached results.

Layer 4: already done, loading cached results.

Layer 5: already done, loading cached results.

Layer 6: already done, loading cached results.

Layer 7: already done, loading cached results.

Layer 8: already done, loading cached results.

Layer 9: already done, loading cached results.

Layer 10: already done, loading cached results.

Layer 11: already done, loading cached results.

Layer 12: already done, loading cached results.

Layer 13: already done, loading cached results.

Layer 14: already done, loading cached results.

Layer 15: already done, loading cached results.

Layer 

In [ ]:
#@title Plot emergence curves across GPT-2 models

import matplotlib.pyplot as plt

df = pd.read_csv(BASE_DIR / "combined_layer_metrics.csv")

plt.figure(figsize=(10,5))
for model_name in MODEL_NAMES:
    sub = df[df["model"] == model_name].sort_values("layer")
    if sub.empty:
        continue
    plt.plot(sub["layer"], sub["R2"], marker="o", label=f"{model_name} (R²)")

plt.xlabel("Layer index")
plt.ylabel("Block R²")
plt.title("Emergence of modular block structure across layers")
plt.grid(True)
plt.legend()
plt.show()

plt.figure(figsize=(10,5))
for model_name in MODEL_NAMES:
    sub = df[df["model"] == model_name].sort_values("layer")
    if sub.empty:
        continue
    plt.plot(sub["layer"], sub["spec_mean"], marker="o", label=f"{model_name} (spec)")

plt.xlabel("Layer index")
plt.ylabel("Mean specialization (1 - H_norm)")
plt.title("Neuron specialization across layers")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
#@title Inspect token clusters for a chosen model & layer

#@markdown Choose a model and layer to inspect. THIS IS SO SICK THAT YOU CAN PUT IN PARAMS LIKE THIS!!!
model_name = "gpt2-xl"  #@param ["gpt2","gpt2-medium","gpt2-large","gpt2-xl"]
layer_to_inspect = 47   #@param {type:"integer"}

from collections import Counter

tokens = np.load(BASE_DIR / "dataset_tokens.npz")["input_ids"]
seqs = make_sequences_from_stream(tokens, SEQ_LEN)

mdir = BASE_DIR / model_name.replace("/", "_")
layer_data = np.load(mdir / f"activations_layer{layer_to_inspect:02d}.npz")["activations"]
cluster_data = np.load(mdir / "clusters" / f"layer{layer_to_inspect:02d}_best_cluster.npz")
row_labels = cluster_data["row_labels"]
col_labels = cluster_data["col_labels"]
row_k = int(cluster_data["row_k"])
col_k = int(cluster_data["col_k"])

print(f"Layer {layer_to_inspect}: row_k={row_k}, col_k={col_k}, activations shape={layer_data.shape}")

N = layer_data.shape[0]
token_ids_sample = tokens[:N]
token_strs_sample = np.array(tokenizer.convert_ids_to_tokens(token_ids_sample.tolist()))

for r in range(row_k):
    idx = np.where(row_labels == r)[0]
    if idx.size == 0:
        continue
    toks = token_strs_sample[idx]
    counts = Counter(toks)
    top = counts.most_common(30)
    print(f"\n=== Token cluster {r} | size={idx.size} ===")
    print("Top tokens:")
    print(", ".join([f"{t}:{c}" for t,c in top]))
